In [4]:
# -*- coding: utf-8 -*-
"""
Merge Antarctic MODIS ObsCount tiles by year.
Input : G:/Hangkai/Anttarctic Vegetation Dynamic/MODIS_OBS/Antarctic_ObsCount
Output: G:/Hangkai/Anttarctic Vegetation Dynamic/MODIS_OBS/Antarctic_ObsCount/Merged
"""

from pathlib import Path
import re
import rasterio
from rasterio.merge import merge

# ================= CONFIG =================
base_dir = Path(r"/mnt/cephfs-mount/hangkai/Antarctic_ObsCount/")
out_dir  = base_dir / "Merged"
out_dir.mkdir(exist_ok=True)

# ================= MAIN =================
# Find all .tif files
tif_files = list(base_dir.glob("*.tif"))

# Extract years using regex (e.g., ObsCount_2002_Sep23_to_Mar21-....tif)
pattern = re.compile(r"ObsCount_(\d{4})_Sep23_to_Mar21")

# Group by year
year_groups = {}
for tif in tif_files:
    match = pattern.search(tif.name)
    if match:
        year = match.group(1)
        year_groups.setdefault(year, []).append(tif)

print(f"Found {len(year_groups)} years:", list(year_groups.keys()))

# Merge per year
for year, files in sorted(year_groups.items()):
    print(f"\nMerging {year} ({len(files)} tiles)...")
    srcs = [rasterio.open(f) for f in files]
    mosaic, transform = merge(srcs)
    meta = srcs[0].meta.copy()
    for s in srcs: s.close()

    meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": transform,
        "compress": "lzw"
    })

    out_path = out_dir / f"ObsCount_{year}_Sep23_to_Mar21_MERGED.tif"
    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(mosaic)
    print(f"Saved: {out_path}")

print("\n✅ All years merged successfully!")


Found 22 years: ['2019', '2017', '2011', '2018', '2015', '2010', '2002', '2007', '2020', '2006', '2013', '2012', '2016', '2021', '2008', '2003', '2004', '2023', '2014', '2022', '2005', '2009']

Merging 2002 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2002_Sep23_to_Mar21_MERGED.tif

Merging 2003 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2003_Sep23_to_Mar21_MERGED.tif

Merging 2004 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2004_Sep23_to_Mar21_MERGED.tif

Merging 2005 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2005_Sep23_to_Mar21_MERGED.tif

Merging 2006 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2006_Sep23_to_Mar21_MERGED.tif

Merging 2007 (9 tiles)...
Saved: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/ObsCount_2007_Sep23_to_Mar21_MERGED.tif

Merging 2008 (9 tiles)...
Saved: /mnt/cephfs-mount

In [ ]:
# -*- coding: utf-8 -*-
"""
Compute mean and standard deviation of clear observations (ObsCount)
across years, and save the maps.

Input : /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged
Output: /mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged/Stats
"""

from pathlib import Path
import numpy as np
import rasterio
import re
import pandas as pd
from rasterio.enums import Resampling

# ================= CONFIG =================
in_dir  = Path("/mnt/cephfs-mount/hangkai/Antarctic_ObsCount/Merged")
out_dir = in_dir / "Stats"
out_dir.mkdir(exist_ok=True)

# ================= LOAD FILES =================
pattern = re.compile(r"ObsCount_(\d{4})_Sep23_to_Mar21_MERGED\.tif$")
tifs = sorted([f for f in in_dir.glob("*.tif") if pattern.search(f.name)])

if not tifs:
    raise FileNotFoundError(f"No merged files found in {in_dir}")

years = []
stack = []
meta = None

for fp in tifs:
    year = int(pattern.search(fp.name).group(1))
    with rasterio.open(fp) as src:
        arr = src.read(1, masked=True)
        if meta is None:
            meta = src.meta.copy()
        stack.append(arr)
        years.append(year)

# Convert to 3D masked array [time, rows, cols]
data = np.ma.stack(stack, axis=0)

# ================= PER-YEAR STATS =================
records = []
for i, year in enumerate(years):
    arr = data[i]
    records.append({
        "year": year,
        "valid_px": np.ma.count(arr),
        "mean_obs": float(arr.mean()),
        "std_obs": float(arr.std())
    })
df = pd.DataFrame(records).sort_values("year")
print(df)

# ================= MEAN & STD MAPS =================
mean_map = np.ma.mean(data, axis=0)
std_map  = np.ma.std(data, axis=0)

# Update metadata
meta.update({
    "driver": "GTiff",
    "dtype": "float32",
    "count": 1,
    "compress": "lzw"
})

# Save mean map
mean_path = out_dir / "ObsCount_Mean_AcrossYears.tif"
with rasterio.open(mean_path, "w", **meta) as dst:
    dst.write(mean_map.filled(np.nan).astype("float32"), 1)
print(f"✅ Saved mean map: {mean_path}")

# Save std map
std_path = out_dir / "ObsCount_Std_AcrossYears.tif"
with rasterio.open(std_path, "w", **meta) as dst:
    dst.write(std_map.filled(np.nan).astype("float32"), 1)
print(f"✅ Saved std map: {std_path}")

# ================= SUMMARY =================
print("\n===== SUMMARY OF CLEAR OBS =====")
print(f"Overall mean of mean-map: {mean_map.mean():.2f}")
print(f"Overall std  of mean-map: {mean_map.std():.2f}")
print(f"Overall mean of std-map : {std_map.mean():.2f}")
print(f"Overall std  of std-map : {std_map.std():.2f}")

# Optional: save yearly summary table
df.to_csv(out_dir / "ObsCount_Yearly_Stats.csv", index=False)
print(f"✅ Saved yearly stats table: {out_dir / 'ObsCount_Yearly_Stats.csv'}")

In [ ]:
# Forest_Depth_ED_Mapping.py
# Local 3x3 neighbor mosaic + halo Euclidean forest depth (meters).
# 1 = forest, 0 = non-forest. Output: float32 meters; non-forest set to 0.
#
# How to run:
#   python Forest_Depth_ED_Mapping.py
#
# Dependencies:
#   pip install rasterio numpy scipy tqdm pyproj
# (pyproj is optional but improves degree->meter accuracy.)
#
# Notes:
# - Works whether rasters are in a projected CRS (meters) or in degrees (EPSG:4326 etc.).
# - For degree CRS, meters-per-pixel is computed using local latitude (pyproj.Geod if available).
# - Uses only the target tile + its 3×3 neighborhood and a pixel halo to avoid global VRTs.

import os, math, time, warnings, gc
from datetime import datetime
from rasterio.windows import Window
import numpy as np
import rasterio
from rasterio.transform import array_bounds
from affine import Affine
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm

# --------- USER SETTINGS (edit these paths if needed) ---------
INPUT_DIR  = r"/mnt/cephfs-mount/hangkai/2020"
OUTPUT_DIR = r"/mnt/cephfs-mount/hangkai/2020_depth"

HALO_PX = 2048                # Pixel halo around each tile (e.g., 256 / 512 / 1024)
SUBTRACT_HALF_PIXEL = True    # Better approximates distance to boundary vs pixel center
OVERWRITE = False             # Skip tiles that already have outputs when False
LIMIT_TILES = None            # e.g., 5 to test only first 5 tiles; set None for all
COMPRESSION = "LZW"           # "LZW" or "DEFLATE" etc.

# --------- Optional: accurate meters-per-degree via pyproj ---------
try:
    from pyproj import Geod
    GEOD = Geod(ellps="WGS84")
except Exception:
    GEOD = None


# ====================== Utilities ======================
def now_str():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def list_tifs(input_dir):
    tifs = [os.path.join(input_dir, n) for n in os.listdir(input_dir)
            if n.lower().endswith((".tif", ".tiff"))]
    tifs.sort()
    return tifs

def build_tile_index(tif_paths):
    """Gather per-tile metadata used for neighbor search and merging."""
    index = []
    crs0 = None
    for p in tqdm(tif_paths, desc="Indexing tiles", unit="tile"):
        with rasterio.open(p) as ds:
            h, w = ds.height, ds.width
            btm, lft, top, rgt = array_bounds(h, w, ds.transform)  # (ymin, xmin, ymax, xmax)
            if abs(ds.transform.b) > 1e-9 or abs(ds.transform.d) > 1e-9:
                print(f"[{now_str()}] WARNING: {os.path.basename(p)} has rotation/shear in transform.")
            if crs0 is None:
                crs0 = ds.crs
            elif ds.crs != crs0:
                print(f"[{now_str()}] WARNING: CRS differs for {os.path.basename(p)}.")
            index.append({
                "path": p,
                "name": os.path.basename(p),
                "width": w,
                "height": h,
                "transform": ds.transform,
                "bounds": (lft, btm, rgt, top),             # (L, B, R, T)
                "center": ((lft+rgt)/2.0, (btm+top)/2.0),   # (cx, cy)
                "xres": abs(ds.transform.a),
                "yres": abs(ds.transform.e),
                "crs": ds.crs
            })
    # Quick lookup by path
    path2meta = {m["path"]: m for m in index}
    return index, path2meta

def intersects(b1, b2):
    L1,B1,R1,T1 = b1
    L2,B2,R2,T2 = b2
    return (L1 < R2) and (R1 > L2) and (B1 < T2) and (T1 > B2)

def meters_per_pixel_from_shape(transform, width, height, crs, center_lat=None):
    """
    Return (yres_m, xres_m).
    If projected (meter) CRS -> return pixel sizes directly (with a crude feet->meters guard).
    If geographic -> convert degree sizes to meters at provided/derived latitude.
    """
    xres = abs(transform.a)
    yres = abs(transform.e)

    if crs and crs.is_projected:
        # crude sniff for feet; avoids ~3.28x inflation if any tile is in US feet
        try:
            wkt = crs.to_wkt().lower()
            if "foot" in wkt or "feet" in wkt or "us-ft" in wkt:
                return (yres * 0.3048, xres * 0.3048)
        except Exception:
            pass
        return (yres, xres)

    # Geographic: need representative latitude
    if center_lat is None:
        btm, lft, top, rgt = array_bounds(height, width, transform)
        center_lat = (top + btm) / 2.0

    if GEOD is not None:
        # Use great-circle distances for 1 px steps
        btm, lft, top, rgt = array_bounds(height, width, transform)
        center_lon = (lft + rgt) / 2.0
        _, _, dx_m = GEOD.inv(center_lon, center_lat, center_lon + xres, center_lat)
        _, _, dy_m = GEOD.inv(center_lon, center_lat, center_lon, center_lat + yres)
        return (abs(dy_m), abs(dx_m))

    # Fallback approximation formulas
    phi = math.radians(center_lat)
    m_per_deg_lat = 111132.92 - 559.82*math.cos(2*phi) + 1.175*math.cos(4*phi) - 0.0023*math.cos(6*phi)
    m_per_deg_lon = 111412.84*math.cos(phi) - 93.5*math.cos(3*phi) + 0.118*math.cos(5*phi)
    return (yres * m_per_deg_lat, xres * m_per_deg_lon)


def process_tile_local3x3(tile_meta, all_meta, halo_px, out_path,
                          subtract_half_pixel=True, compress="LZW"):
    """
    Forest depth for one tile using a 3×3 neighborhood with integer-grid placement (no resampling).

    Assumptions
    -----------
    - All tiles share CRS, pixel size, and are north-up (no rotation/shear).
    - Tiles are perfectly aligned on the same grid.
    """
    t0 = time.perf_counter()

    # ---- Center tile basics ----
    W, H = tile_meta["width"], tile_meta["height"]
    tx, crs = tile_meta["transform"], tile_meta["crs"]

    # Safety: north-up only
    if abs(tx.b) > 1e-9 or abs(tx.d) > 1e-9:
        raise RuntimeError("Center tile has rotation/shear; integer-grid placement requires north-up.")

    a = tx.a                   # col scale (>0)
    e = tx.e                   # row scale (<0 for north-up)
    xres = abs(a)
    yres = abs(e)

    # ---- Target grid (expanded outward by HALO_PX) ----
    DST_W  = W + 2*halo_px
    DST_H  = H + 2*halo_px
    dst_transform = tx * Affine.translation(-halo_px, -halo_px)

    # Target mask: 0 = non-forest, 1 = forest
    mosaic_u8 = np.zeros((DST_H, DST_W), dtype=np.uint8)

    # ---- Load center tile once (for final mask + profile; short-circuit if empty) ----
    with rasterio.open(tile_meta["path"]) as src_center:
        center_profile = src_center.profile
        center_arr = src_center.read(1)

    center_has_forest = bool((center_arr == 1).any())
    if not center_has_forest:
        # Nothing to compute—write zeros and return
        core = np.zeros((H, W), dtype=np.float32)
        center_profile.update(
            driver="GTiff", dtype="float32", count=1, nodata=0.0,
            compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
            bigtiff="IF_SAFER", interleave="band"
        )
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with rasterio.open(out_path, "w", **center_profile) as dst:
            dst.write(core, 1)
        del center_arr, core
        gc.collect()
        return time.perf_counter() - t0

    # ---- Map neighbors to grid offsets using upper-left coordinates ----
    cx0, cy0 = tx.c, tx.f  # UL of center tile

    grid = {}
    for m in all_meta:
        mt = m["transform"]
        # Sanity checks
        if m["crs"] != crs: continue
        if abs(mt.b) > 1e-9 or abs(mt.d) > 1e-9: continue  # rotated -> skip
        if abs(mt.a - a) > 1e-9 or abs(mt.e - e) > 1e-9: continue  # different pixel size

        nx0, ny0 = mt.c, mt.f  # neighbor UL world coords
        dc = (nx0 - cx0) / a   # columns offset (float)
        dr = (ny0 - cy0) / e   # rows offset (float)  (e<0)

        dx_step = int(round(dc / W))
        dy_step = int(round(dr / H))
        if dx_step < -1 or dx_step > 1 or dy_step < -1 or dy_step > 1:
            continue

        key = (dy_step, dx_step)
        if key not in grid:
            grid[key] = m

    # Ensure center tile itself exists
    grid.setdefault((0, 0), tile_meta)

    # ---- Paste each neighbor onto the target canvas (integer indices only) ----
    def paste_neighbor_exact(src_path, dy_step, dx_step):
        """
        Place one neighbor on the target canvas by integer offsets.
        """
        # Destination top-left for this neighbor in the mosaic (in pixels)
        dr0 = halo_px + dy_step * H    # rows
        dc0 = halo_px + dx_step * W    # cols

        # If completely outside the canvas, skip
        if dr0 >= DST_H or dc0 >= DST_W or dr0 + H <= 0 or dc0 + W <= 0:
            return

        # Source start offsets if the neighbor starts outside the canvas
        sr0 = max(0, -dr0)             # rows to skip at source top
        sc0 = max(0, -dc0)             # cols to skip at source left
        dr0c = max(0, dr0)             # clamped dest row
        dc0c = max(0, dc0)             # clamped dest col

        # Intended size
        hh = min(H - sr0, DST_H - dr0c)   # rows (height)
        ww = min(W - sc0, DST_W - dc0c)   # cols  (width)
        if hh <= 0 or ww <= 0:
            return

        with rasterio.open(src_path, sharing=False) as src:
            # FIXED: Window order is (col_off, row_off, width, height)
            patch = src.read(1, window=Window(sc0, sr0, ww, hh))

        # Strict 0/1 forest mask
        patch = (patch == 1).astype(np.uint8)

        # Destination view
        view = mosaic_u8[dr0c:dr0c+hh, dc0c:dc0c+ww]

        # Shapes must match; clip if necessary (should rarely be needed)
        if patch.shape != view.shape:
            h_eff = min(patch.shape[0], view.shape[0])
            w_eff = min(patch.shape[1], view.shape[1])
            patch = patch[:h_eff, :w_eff]
            view  = mosaic_u8[dr0c:dr0c+h_eff, dc0c:dc0c+w_eff]

        np.maximum(view, patch, out=view)

    # Paste center first, then neighbors so center wins on overlaps
    order = [(0,0), (-1,0),(+1,0),(0,-1),(0,+1),(-1,-1),(-1,+1),(+1,-1),(+1,+1)]
    for dy_step, dx_step in order:
        m = grid.get((dy_step, dx_step))
        if not m:
            continue
        paste_neighbor_exact(m["path"], dy_step, dx_step)

    # ---- EDT on merged mask ----
    # Pixel spacing (meters) from the target grid
    ymin, xmin, ymax, xmax = array_bounds(DST_H, DST_W, dst_transform)
    center_lat = (ymax + ymin) / 2.0
    yres_m, xres_m = meters_per_pixel_from_shape(dst_transform, DST_W, DST_H, crs, center_lat)

    forest_mask = (mosaic_u8 == 1)
    del mosaic_u8; gc.collect()

    # GUARD: if no background (all forest in 3×3 + halo), write zeros and exit
    if not np.any(~forest_mask):
        core = np.zeros((H, W), dtype=np.float32)
        profile = center_profile
        profile.update(
            driver="GTiff", dtype="float32", count=1, nodata=0.0,
            compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
            bigtiff="IF_SAFER", interleave="band"
        )
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(core, 1)
        del forest_mask, core
        gc.collect()
        return time.perf_counter() - t0

    # Distance transform with anisotropic sampling (meters)
    dt64 = distance_transform_edt(forest_mask, sampling=(yres_m, xres_m))
    dist_chunk = dt64.astype(np.float32)
    del dt64, forest_mask; gc.collect()

    if subtract_half_pixel:
        # Boundary (not center-to-center) distance
        half_min = 0.5 * min(yres_m, xres_m)
        np.maximum(dist_chunk - half_min, 0.0, out=dist_chunk)

    # ---- Crop the center tile core (exactly H×W at (halo,halo)) ----
    core = dist_chunk[HALO_PX:HALO_PX+H, HALO_PX:HALO_PX+W]
    assert core.shape == (H, W)
    del dist_chunk; gc.collect()

    # ---- Final mask: keep values only where center tile itself is forest ----
    core = np.where(center_arr == 1, core, 0.0).astype(np.float32)
    del center_arr; gc.collect()

    # ---- Sanitize: force NaN/Inf/nodata to 0, clamp to [0, MAX] (optional clamp) ----
    MAX_VAL = 1_000_000.0
    core = np.nan_to_num(core, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32, copy=False)
    np.clip(core, 0.0, MAX_VAL, out=core)

    # ---- Write single-band float32 GTiff (LZW + predictor=3) with nodata=0 ----
    profile = center_profile
    profile.update(
        driver="GTiff", dtype="float32", count=1, nodata=0.0,
        compress=compress, predictor=3, tiled=True, blockxsize=512, blockysize=512,
        bigtiff="IF_SAFER", interleave="band"
    )
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(core, 1)

    del core
    gc.collect()
    return time.perf_counter() - t0


# ====================== Main Routine ======================
def main():
    os.environ.setdefault("GDAL_PAM_ENABLED", "NO")  # no .aux.xml
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    tif_list = list_tifs(INPUT_DIR)
    if not tif_list:
        print(f"[{now_str()}] No GeoTIFFs found in: {INPUT_DIR}")
        return

    if LIMIT_TILES is not None:
        tif_list = tif_list[:int(LIMIT_TILES)]

    print(f"[{now_str()}] Mode: LOCAL 3x3 + HALO")
    print(f"  Input : {INPUT_DIR}")
    print(f"  Output: {OUTPUT_DIR}")
    print(f"  Files : {len(tif_list)}")
    print(f"  Halo (px): {HALO_PX}")
    print(f"  Subtract half pixel: {SUBTRACT_HALF_PIXEL}\n")

    # Build metadata index once
    index, path2meta = build_tile_index(tif_list)

    total_start = time.perf_counter()
    failures = 0

    for p in tqdm(tif_list, desc="Processing tiles", unit="tile"):
        name = os.path.basename(p)
        out_name = os.path.splitext(name)[0] + "_depth_m.tif"
        out_path = os.path.join(OUTPUT_DIR, out_name)

        if (not OVERWRITE) and os.path.exists(out_path):
            tqdm.write(f"[{now_str()}] Skip (exists): {out_name}")
            continue

        try:
            tile_meta = path2meta[p]
            dt = process_tile_local3x3(
                tile_meta, index, HALO_PX, out_path,
                subtract_half_pixel=SUBTRACT_HALF_PIXEL,
                compress=COMPRESSION
            )
            tqdm.write(f"[{now_str()}] Done {name} in {dt:.2f} s")
        except Exception as e:
            failures += 1
            tqdm.write(f"[{now_str()}] ERROR {name}: {e}")

    total_s = time.perf_counter() - total_start
    print(f"\n[{now_str()}] All done in {total_s/60.0:.2f} min. Failures: {failures}")


if __name__ == "__main__":
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        main()

[2025-09-18 03:12:46] Mode: LOCAL 3x3 + HALO
  Input : /mnt/cephfs-mount/hangkai/2020
  Output: /mnt/cephfs-mount/hangkai/2020_depth
  Files : 259
  Halo (px): 2048
  Subtract half pixel: True



Processing tiles:  14%|███                   | 36/259 [00:00<00:01, 175.44tile/s]

[2025-09-18 03:12:48] Skip (exists): 00N_000E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_010E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_020E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_030E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_040E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_040W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_050W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_060W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_070W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_080W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_090E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_090W_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_100E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_110E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_120E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_130E_depth_m.tif
[2025-09-18 03:12:48] Skip (exists): 00N_140E_depth_m.tif
[2025-09-18 03

Processing tiles:  28%|██████                | 72/259 [00:00<00:01, 174.61tile/s]

[2025-09-18 03:12:49] Skip (exists): 10N_120E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10N_130E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_010E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_020E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_030E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_040E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_040W_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_050E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_050W_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_060W_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_070W_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_080W_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_110E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_120E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_130E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_140E_depth_m.tif
[2025-09-18 03:12:49] Skip (exists): 10S_150E_depth_m.tif
[2025-09-18 03

...

[2025-09-18 05:08:04] Done 60N_160W.tif in 264.63 s


Processing tiles:  81%|█████████████▊   | 211/259 [1:59:19<3:23:41, 254.62s/tile]

[2025-09-18 05:12:08] Done 60N_170E.tif in 243.91 s


Processing tiles:  82%|█████████████▉   | 212/259 [2:03:31<3:18:51, 253.87s/tile]

[2025-09-18 05:16:20] Done 60N_170W.tif in 252.11 s


In [2]:
import os
import zipfile
from pathlib import Path
from tqdm import tqdm

# === Paths ===
base_dir = Path("/mnt/cephfs-mount/chenchen/ERA5_Climate_Data")
output_base = base_dir / "unzip"
output_base.mkdir(exist_ok=True)

# === Find all .zip files ===
zip_files = sorted(base_dir.glob("*.zip"))

print(f"Found {len(zip_files)} zip files.")

# === Unzip loop ===
for zip_path in tqdm(zip_files, desc="Unzipping files"):
    # e.g. ERA5Land_2018_08_daily_mean.zip → ERA5Land_2018_08
    subfolder_name = "_".join(zip_path.stem.split("_")[:3])
    output_folder = output_base / subfolder_name

    # Skip if already unzipped
    if output_folder.exists() and any(output_folder.iterdir()):
        print(f"⏩ Skipping {subfolder_name} (already unzipped)")
        continue

    output_folder.mkdir(parents=True, exist_ok=True)

    # Unzip content
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(output_folder)

print("✅ All files processed successfully.")

Found 47 zip files.


Unzipping files:   0%|                                                                                | 0/47 [00:00<?, ?it/s]

⏩ Skipping ERA5Land_2018_08 (already unzipped)
⏩ Skipping ERA5Land_2018_09 (already unzipped)
⏩ Skipping ERA5Land_2018_10 (already unzipped)
⏩ Skipping ERA5Land_2018_11 (already unzipped)
⏩ Skipping ERA5Land_2018_12 (already unzipped)
⏩ Skipping ERA5Land_2019_01 (already unzipped)
⏩ Skipping ERA5Land_2019_02 (already unzipped)
⏩ Skipping ERA5Land_2019_03 (already unzipped)
⏩ Skipping ERA5Land_2019_04 (already unzipped)
⏩ Skipping ERA5Land_2019_05 (already unzipped)
⏩ Skipping ERA5Land_2019_06 (already unzipped)
⏩ Skipping ERA5Land_2019_07 (already unzipped)
⏩ Skipping ERA5Land_2019_08 (already unzipped)
⏩ Skipping ERA5Land_2019_09 (already unzipped)
⏩ Skipping ERA5Land_2019_10 (already unzipped)


Unzipping files: 100%|███████████████████████████████████████████████████████████████████████| 47/47 [00:03<00:00, 12.31it/s]

⏩ Skipping ERA5Land_2021_01 (already unzipped)
⏩ Skipping ERA5Land_2021_02 (already unzipped)
⏩ Skipping ERA5Land_2021_03 (already unzipped)
⏩ Skipping ERA5Land_2021_04 (already unzipped)
⏩ Skipping ERA5Land_2021_05 (already unzipped)
⏩ Skipping ERA5Land_2021_06 (already unzipped)
⏩ Skipping ERA5Land_2021_07 (already unzipped)
⏩ Skipping ERA5Land_2021_08 (already unzipped)
⏩ Skipping ERA5Land_2021_09 (already unzipped)
⏩ Skipping ERA5Land_2021_10 (already unzipped)
⏩ Skipping ERA5Land_2021_11 (already unzipped)
⏩ Skipping ERA5Land_2021_12 (already unzipped)
⏩ Skipping ERA5Land_2022_01 (already unzipped)
⏩ Skipping ERA5Land_2022_02 (already unzipped)
⏩ Skipping ERA5Land_2022_03 (already unzipped)
⏩ Skipping ERA5Land_2022_04 (already unzipped)
⏩ Skipping ERA5Land_2022_05 (already unzipped)
⏩ Skipping ERA5Land_2022_06 (already unzipped)
⏩ Skipping ERA5Land_2022_07 (already unzipped)
⏩ Skipping ERA5Land_2022_08 (already unzipped)
⏩ Skipping ERA5Land_2022_09 (already unzipped)
⏩ Skipping ER

In [ ]:
import os
import numpy as np
import rasterio
from geopy.distance import geodesic
import gc
from tqdm import tqdm

# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters

# Function to compute forest edge length
def compute_edge_length(i, j, forest, width_lookup, height):
    edge_length = 0.0  # Initialize edge length

    # Check left pixel
    if j > 0 and not forest[i, j-1]:
        edge_length += height  # Add height to edge length

    # Check right pixel
    if j < forest.shape[1] - 1 and not forest[i, j+1]:
        edge_length += height  # Add height to edge length

    # Check upper pixel
    if i > 0 and not forest[i-1, j]:
        edge_length += width_lookup[i-1]  # Add width of upper pixel to edge length

    # Check lower pixel
    if i < forest.shape[0] - 1 and not forest[i+1, j]:
        edge_length += width_lookup[i]  # Add width of current pixel to edge length

    return edge_length

# Function to process a single TIFF file
def process_tiff(filename):
    # Full paths to input and output files
    input_file = os.path.join(input_folder, filename)
    output_file_area = os.path.join(output_folder_area, filename)
    output_file_edge = os.path.join(output_folder_edge, filename)
    #print(f"Processing file: {filename}")

    # Open tif file
    with rasterio.open(input_file) as ds:
        forest = ds.read(1).astype(bool)
        transform = ds.transform
        cellsize = abs(transform[0])
        #print(f"Cellsize: {cellsize}")

        # Compute lookup table for pixel widths and pixel height
        width_lookup = compute_width_lookup(forest.shape[0], transform)
        height = compute_height(cellsize)
        #print("Computed lookup table for pixel widths and pixel height")

        # Create new tifs
        area_tif = np.zeros_like(forest, dtype=np.float32)
        edge_tif = np.zeros_like(forest, dtype=np.float32)
        #print("Created new tifs")

        # Loop through each pixel in the forest tif
        for i in range(forest.shape[0]):
            for j in range(forest.shape[1]):
                # Check if the pixel is a forest pixel
                if forest[i, j]:
                    # Calculate the area of the pixel and assign it to area_tif
                    area_tif[i, j] = width_lookup[i] * height

                    # Calculate the edge length of the pixel and assign it to edge_tif
                    edge_tif[i, j] = compute_edge_length(i, j, forest, width_lookup, height)

        # Convert to unsigned integers with rounding
        scale_factor = 10
        area_tif = np.around(area_tif * scale_factor).astype(np.uint16)
        edge_tif = np.around(edge_tif * scale_factor).astype(np.uint16)

        # Write new tif to file
        with rasterio.open(output_file_area, 'w', driver='GTiff', height=area_tif.shape[0],
                           width=area_tif.shape[1], count=1, compress='lzw', dtype=area_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(area_tif, 1)

        with rasterio.open(output_file_edge, 'w', driver='GTiff', height=edge_tif.shape[0],
                           width=edge_tif.shape[1], count=1, compress='lzw', dtype=edge_tif.dtype,
                           crs=ds.crs, transform=transform) as dst:
            dst.write(edge_tif, 1)

    print(f"Finished processing file: {filename}")
    del forest
    del area_tif
    del edge_tif
    gc.collect()  # Force garbage collection

# Paths to input and output folders
input_folder = "/mnt/cephfs-mount/hangkai/2015/"
output_folder_area = "/mnt/cephfs-mount/hangkai/2015Area"
output_folder_edge = "/mnt/cephfs-mount/hangkai/2015Edge"

# Create output folders if they don't exist
if not os.path.exists(output_folder_area):
    os.makedirs(output_folder_area)
if not os.path.exists(output_folder_edge):
    os.makedirs(output_folder_edge)

# Get list of tif files
tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]
print("Start to loop!")
# Loop over each file in the list
for filename in tqdm(tif_files):
    process_tiff(filename)

In [ ]:
import os
import rasterio
import re
import numpy as np
import copy
from geopy.distance import geodesic
import gc
from tqdm import tqdm

################################################################################################################################
def get_surrounding_files(file_name):
    """Given a file name, return the file names of the files corresponding to 
    the geographical locations to the north, south, east, and west."""

    def extract_first_digit(string):
        """Extract the first sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        match = re.search(pattern, string)
        return int(match.group()) if match else None

    def extract_second_digit(string):
        """Extract the second sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        matches = re.findall(pattern, string)
        return int(matches[1]) if len(matches) >= 2 else None

    def extract_first_letter(string):
        """Extract the first English letter in the string."""
        for char in string:
            if char.isalpha(): return char

    def extract_second_letter(string):
        """Extract the second English letter in the string."""
        count = 0
        for char in string:
            if char.isalpha(): 
                count += 1
                if count == 2: return char

    first_digit = extract_first_digit(file_name)
    second_digit = extract_second_digit(file_name)
    first_letter = extract_first_letter(file_name)
    second_letter = extract_second_letter(file_name)

    if first_letter == 'N':
        up_first_digit = int(first_digit) + 10
        up_first_letter = 'N'
        if first_digit == 0:
            up_first_digit = 10
            up_first_letter = 'N'
            down_first_digit = 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = first_digit + 10
            down_first_digit = int(first_digit) - 10
            down_first_letter = 'N'
            up_first_letter = 'N'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
    elif first_letter == 'S':
        if first_digit == 10:
            up_first_digit = 0
            up_first_letter = 'N'
            down_first_digit = 20
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = int(first_digit) - 10
            up_first_letter = 'S'
            down_first_digit = int(first_digit) + 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter

    # East or West
    if second_letter == 'E':
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = second_digit - 10
        left_second_letter = second_letter
        if second_digit == 0:
            right_second_digit = second_digit + 10
            right_second_letter = second_letter
            left_second_digit = 10
            left_second_letter = 'W'
        if second_digit == 170:
            right_second_digit = 180
            right_second_letter = 'W'
            left_second_digit = second_digit - 10
            left_second_letter = 'E'
    elif second_letter == 'W':
        right_second_digit = second_digit - 10
        right_second_letter = second_letter
        left_second_digit = second_digit + 10
        left_second_letter = second_letter
        if second_digit == 10:
            right_second_digit = 0
            right_second_letter = 'E'
            left_second_digit = 20
            left_second_letter = 'W'
        if second_digit == 180:
            right_second_digit = 170
            right_second_letter = 'W'
            left_second_digit = 170
            left_second_letter = 'E'
        
    north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
    south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
    east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
    west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

    return north_file_name, south_file_name, east_file_name, west_file_name

################################################################################################################################


def get_border_pixels(file_name):
    with rasterio.open(file_name) as dataset:
        band1 = dataset.read(1)  # assuming you want to read the first band

        north_pixels = band1[0, :]  # North border pixels
        south_pixels = band1[-1, :]  # South border pixels
        east_pixels = band1[:, -1]  # East border pixels
        west_pixels = band1[:, 0]  # West border pixels

    return north_pixels, south_pixels, east_pixels, west_pixels

def process_files(input_folder, central_file):
    # Calculate file names of the surrounding files
    north_file, south_file, east_file, west_file = get_surrounding_files(central_file)

    # List of files and corresponding border pixels
    file_list = [(north_file, 1), (south_file, 0), (east_file, 3), (west_file, 2)]

    # Dictionary to store file existence flags and border pixels
    border_data = {}

    # Get the border pixels of the central file
    central_file_path = os.path.join(input_folder, central_file)
    if os.path.exists(central_file_path):
        central_pixels = get_border_pixels(central_file_path)
    else:
        print(f"Warning: {central_file} does not exist. Skipping this file.")
        return None

    for file, border_index in file_list:
        file_path = os.path.join(input_folder, file)
        # Check if the file exists
        if os.path.exists(file_path):
            # For north and south files, we need to consider the south and north borders of the central file, respectively.
            if border_index in [0, 1]:
                central_border_pixels = central_pixels[1 - border_index]  
            # For east and west files, we need to consider the west and east borders of the central file, respectively.
            else:  
                central_border_pixels = central_pixels[5 - border_index]
            border_data[file] = {
                "exists": True, 
                "pixels": get_border_pixels(file_path)[border_index], 
                "central_pixels": central_border_pixels
            }
        else:
            border_data[file] = {"exists": False}
    return border_data

################################################################################################################################

def read_border_pixels(file_path):
    dataset = rasterio.open(file_path)

    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################

def read_border_pixels_edge(file_path):
    dataset = rasterio.open(file_path)
    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################
# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters
################################################################################################################################


# Paths for your folders
input_folder = "/mnt/cephfs-mount/hangkai/2015"
output_folder_edge = "/mnt/cephfs-mount/hangkai/2015Edge"
input_folder_edge = output_folder_edge
# Get all files in each folder
central_files = os.listdir(input_folder)
central_files_edge = os.listdir(output_folder_edge)

# Ensuring that the files in both folders correspond to each other
assert set(central_files) == set(central_files_edge)

print('All functions loaded! Start to correct all the tifs.')

# Loop through each file
for central_file in tqdm(central_files):
    # Ensure the file is a .tif file
    if central_file.endswith('.tif'):
        #print(f"Processing file: {central_file}")

################################################################################################################################
################################################################################################################################

        # Your processing code
        # replace 'process_files', 'read_border_pixels' and 'read_border_pixels_edge' with your actual functions
        # Usage:

        border_data = process_files(input_folder, central_file)
        north_file, south_file, east_file, west_file = get_surrounding_files(central_file)
        central_file_path = os.path.join(input_folder, central_file)
        left_pixels, right_pixels, upper_pixels, lower_pixels = read_border_pixels(central_file_path)

        central_file_path_edge = os.path.join(input_folder_edge, central_file)
        left_pixels_edge, right_pixels_edge, upper_pixels_edge, lower_pixels_edge = read_border_pixels_edge(central_file_path_edge)
        left_pixels_edge_new = copy.copy(left_pixels_edge)
        right_pixels_edge_new = copy.copy(right_pixels_edge)
        upper_pixels_edge_new = copy.copy(upper_pixels_edge)
        lower_pixels_edge_new = copy.copy(lower_pixels_edge)

        # Calculate the width of the top and bottom row of pixels
        central_file_path_edge = os.path.join(input_folder_edge, central_file)
        dataset = rasterio.open(central_file_path)
        width = dataset.width
        height = dataset.height
        width_lookup = compute_width_lookup(dataset.height, dataset.transform)
        top_row_width = width_lookup[0]
        bottom_row_width = width_lookup[-1]

        # Calculate the height of the left and right columns of pixels
        left_column_height = compute_height(abs(dataset.transform[4]))
        right_column_height = compute_height(abs(dataset.transform[0]))


        if border_data[north_file]['exists']:
            for i in range(len(border_data[north_file]['pixels'])):
                b = upper_pixels[i]
                if b == 0:
                    continue
                a = (border_data[north_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    upper_pixels_edge_new[i] = upper_pixels_edge[i] + top_row_width
        if border_data[south_file]['exists']:
            for i in range(len(border_data[south_file]['pixels'])):
                b = lower_pixels[i]
                if b == 0:
                    continue
                a = (border_data[south_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    lower_pixels_edge_new[i] = lower_pixels_edge[i] + bottom_row_width
        if border_data[west_file]['exists']:
            for i in range(len(border_data[west_file]['pixels'])):
                b = left_pixels[i] 
                if b == 0:
                    continue
                a = (border_data[west_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    left_pixels_edge_new[i] = left_pixels_edge[i] + left_column_height
        if border_data[east_file]['exists']:
            for i in range(len(border_data[east_file]['pixels'])):
                b = right_pixels[i]
                if b == 0:
                    continue
                a = (border_data[east_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    right_pixels_edge_new[i] = right_pixels_edge[i] + right_column_height

        with rasterio.open(central_file_path_edge, 'r+') as src:
            # Read the data
            data = src.read(1)

            # Replace the border
            data[0, :] = upper_pixels_edge_new* 10
            data[-1, :] = lower_pixels_edge_new* 10
            data[:, 0] = left_pixels_edge_new* 10
            data[:, -1] = right_pixels_edge_new* 10

            # Convert the data to np.uint16
            data = data.astype(np.uint16)

            # Write the data
            src.write(data, 1)
            del data
            gc.collect()